In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Veri klasörlerinin yolu
normal_klasoru = "/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Raisin/Sağlıklı"
okra_klasoru = "/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/1024x1024 Boyutlu Dosyalar/Stem_Okrali_Grafikler_Siyah_Beyaz_1024x1024"

# Okra ve normal verilerini saklamak için boş liste oluştur
okra_verileri = []
normal_verileri = []

# Okra dosyalarını oku ve verileri okra_verileri listesine ekle
for dosya in os.listdir(okra_klasoru):
    if dosya.endswith('_a.csv'):
        dosya_yolu = os.path.join(okra_klasoru, dosya)
        veri = pd.read_csv(dosya_yolu)
        okra_verileri.append(veri[['Wavelength (nm)', 'Absorbance (AU)']].values.flatten())  # Düzleştirme işlemi

# Normal dosyaları oku ve verileri normal_verileri listesine ekle
for dosya in os.listdir(normal_klasoru):
    if dosya.endswith('_a.csv'):
        dosya_yolu = os.path.join(normal_klasoru, dosya)
        veri = pd.read_csv(dosya_yolu)
        normal_verileri.append(veri[['Wavelength (nm)', 'Absorbance (AU)']].values.flatten())  # Düzleştirme işlemi

# Okra ve normal verilerini birleştir
okra_etiketleri = [1] * len(okra_verileri)  # Okra için etiketler 1
normal_etiketleri = [0] * len(normal_verileri)  # Normal için etiketler 0

# Verileri ve etiketleri birleştir
X = np.concatenate((okra_verileri, normal_verileri))
y = np.concatenate((okra_etiketleri, normal_etiketleri))

# Verileri standartlaştırma
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Verileri eğitim ve test setlerine ayır
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LSTM modeli oluşturma
model = Sequential([
    LSTM(64, input_shape=(X_egitim.shape[1:]), return_sequences=True),
    Flatten(),
    Dense(1, activation='sigmoid')
])

# Modeli derleme
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Hiperparametre aralıklarını tanımlama
param_grid = {
	'hidden_layer_sizes': [(100,), (100, 50), (150, 100)],
	'activation': ['relu', 'tanh', 'sigmoid', 'softmax'],
	'solver': ['adam', 'sgd'],
	'max_iter': [500, 1000, 1500]
}

# GridSearchCV ile en iyi hiperparametreleri bulma
grid_search = GridSearchCV(model, param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_egitim, y_egitim)

# En iyi modeli seçme
model = grid_search.best_estimator_

# Modeli eğitme
model.fit(X_egitim, y_egitim, epochs=20, batch_size=32, validation_split=0.2)

# Modeli değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test setinde kayıp: {loss}")
print(f"Test setinde doğruluk: {accuracy}")


KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Veri klasörlerinin yolu
normal_klasoru = "/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Raisin/Sağlıklı"
okra_klasoru = "/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Raisin/Okralı"

# Okra ve normal verilerini saklamak için boş liste oluştur
okra_verileri = []
normal_verileri = []

# Okra dosyalarını oku ve verileri okra_verileri listesine ekle
for dosya in os.listdir(okra_klasoru):
    if dosya.endswith('_a.csv'):
        dosya_yolu = os.path.join(okra_klasoru, dosya)
        veri = pd.read_csv(dosya_yolu)
        okra_verileri.append(veri[['Wavelength (nm)', 'Absorbance (AU)']].values.flatten())  # Düzleştirme işlemi

# Normal dosyaları oku ve verileri normal_verileri listesine ekle
for dosya in os.listdir(normal_klasoru):
    if dosya.endswith('_r.csv'):
        dosya_yolu = os.path.join(normal_klasoru, dosya)
        veri = pd.read_csv(dosya_yolu)
        normal_verileri.append(veri[['Wavelength (nm)', 'Absorbance (AU)']].values.flatten())  # Düzleştirme işlemi

# Okra ve normal verilerini birleştir
okra_etiketleri = [1] * len(okra_verileri)  # Okra için etiketler 1
normal_etiketleri = [0] * len(normal_verileri)  # Normal için etiketler 0

# Verileri ve etiketleri birleştir
X = np.concatenate((okra_verileri, normal_verileri))
y = np.concatenate((okra_etiketleri, normal_etiketleri))

# Verileri standartlaştırma
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Verileri eğitim ve test setlerine ayır
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LSTM modeli oluşturma
model = Sequential([
    LSTM(128, input_shape=(X_egitim.shape[1:],), return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Modeli derleme
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Hiperparametre aralıklarını tanımlama
param_grid = {
    'batch_size': [32, 64],
    'epochs': [10, 20],
}

# GridSearchCV ile en iyi hiperparametreleri bulma
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(X_egitim, y_egitim)

# En iyi modeli seçme
best_model = grid_result.best_estimator_

# Modeli eğitme
best_model.fit(X_egitim, y_egitim)

# Modeli değerlendirme
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f"Test setinde kayıp: {loss}")
print(f"Test setinde doğruluk: {accuracy}")


ValueError: Found array with dim 3. StandardScaler expected <= 2.

#LSTM With Cross Validation, Dropout + Early Stopping PNG görselleri ile eğitim

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Google Drive'ı bağlayın
from google.colab import drive
drive.mount('/content/drive')

# Veri klasörlerinin yolu
normal_klasoru = "/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Manually_Splitted_Datas/400Data_Train/Sağlıklı_Çift_512x512"
okra_klasoru = "/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Manually_Splitted_Datas/400Data_Train/Okralı_Çift_512x512"

# Okra ve normal verilerini saklamak için boş liste oluştur
okra_gorselleri = []
normal_gorselleri = []

# Okra dosyalarını oku ve gorselleri okra_gorselleri listesine ekle
for dosya in os.listdir(okra_klasoru):
    if dosya.endswith('.png'):
        dosya_yolu = os.path.join(okra_klasoru, dosya)
        gorsel = load_img(dosya_yolu, color_mode='grayscale', target_size=(512,512))
        gorsel = img_to_array(gorsel)
        okra_gorselleri.append(gorsel)

# Normal dosyaları oku ve gorselleri normal_gorselleri listesine ekle
for dosya in os.listdir(normal_klasoru):
    if dosya.endswith('.png'):
        dosya_yolu = os.path.join(normal_klasoru, dosya)
        gorsel = load_img(dosya_yolu, color_mode='grayscale', target_size=(512,512))
        gorsel = img_to_array(gorsel)
        normal_gorselleri.append(gorsel)

# Okra ve normal gorsellerini birleştir
okra_gorselleri = np.array(okra_gorselleri)
normal_gorselleri = np.array(normal_gorselleri)
X = np.concatenate((okra_gorselleri, normal_gorselleri))

# Etiketleri oluştur
okra_etiketleri = np.ones(len(okra_gorselleri))
normal_etiketleri = np.zeros(len(normal_gorselleri))
y = np.concatenate((okra_etiketleri, normal_etiketleri))

# Verileri standartlaştırma
scaler = StandardScaler()
X = scaler.fit_transform(X.reshape(X.shape[0], -1))
X = X.reshape(X.shape[0], 1, 512*512)  # LSTM giriş şekline uyacak şekilde yeniden şekillendir

# Verileri eğitim ve test setlerine ayırma
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.model_selection import KFold

# K-fold cross validation için k değerini belirleyin
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Modelinizi her bir fold için eğitin ve değerlendirin
for i, (train_index, val_index) in enumerate(kf.split(X)):
    print(f"Fold {i+1}/{k}")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Modeli yeniden oluşturun (her fold'da sıfırlamak önemlidir)
    model = Sequential([
        LSTM(128, input_shape=(1, 512*512), return_sequences=True),
        Dropout(0.5),
        LSTM(64, return_sequences=False),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    # Özel bir learning rate ile Adam optimizer oluşturma
    optimizer = Adam(learning_rate=0.00005)

    # Modeli derleyin
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping tanımlama
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Modeli eğitme ve early stopping kullanma
    model.fit(X_train, y_train, epochs=40, batch_size=16, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Sonuçları ortalama olarak değerlendirin
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test setinde kayıp: {loss}")
print(f"Test setinde doğruluk: {accuracy}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fold 1/5
Epoch 1/40
40/40 [==============================] - 13s 165ms/step - loss: 0.6763 - accuracy: 0.6019 - val_loss: 0.6609 - val_accuracy: 0.6500
Epoch 2/40
40/40 [==============================] - 4s 107ms/step - loss: 0.6459 - accuracy: 0.6552 - val_loss: 0.6482 - val_accuracy: 0.6562
Epoch 3/40
40/40 [==============================] - 5s 130ms/step - loss: 0.6249 - accuracy: 0.6803 - val_loss: 0.6436 - val_accuracy: 0.6625
Epoch 4/40
40/40 [==============================] - 4s 111ms/step - loss: 0.6045 - accuracy: 0.6975 - val_loss: 0.6354 - val_accuracy: 0.6750
Epoch 5/40
40/40 [==============================] - 4s 105ms/step - loss: 0.5820 - accuracy: 0.7210 - val_loss: 0.6336 - val_accuracy: 0.6562
Epoch 6/40
40/40 [==============================] - 4s 105ms/step - loss: 0.5728 - accuracy: 0.7132 - val_loss: 0.6331 - val_accuracy: 0.6750
Epoch 7/4

#MODEL KAYDETME

In [ ]:
from tensorflow.keras.models import load_model
model.save("/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/saved_model/400TrainDataDeneme_LSTM(Cross_Dropout_Early)_Model.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# LSTM PNG With Cross Validation, Dropout, Blurriness + Early Stopping

In [ ]:
import os
import numpy as np
from scipy.ndimage import gaussian_filter
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Google Drive'ı bağlayın
from google.colab import drive
drive.mount('/content/drive')

# Veri klasörlerinin yolu
normal_klasoru = "/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Sağlıklı_Çift_Grafik_512x512"
okra_klasoru = "/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Okralı_Çift_Grafik_512x512"

# Okra ve normal verilerini saklamak için boş liste oluştur
okra_gorselleri = []
normal_gorselleri = []

# Fonksiyon: Görüntü yükleme ve bulanıklık ekleme
def load_and_blur_image(image_path, target_size=(512,512)):
    img = load_img(image_path, color_mode='grayscale', target_size=target_size)
    img_array = img_to_array(img)
    blurred_img_array = gaussian_filter(img_array, sigma=1.5)  # Bulanıklık ekleme
    return blurred_img_array

# Okra dosyalarını oku ve gorselleri okra_gorselleri listesine ekle
for dosya in os.listdir(okra_klasoru):
    if dosya.endswith('.png'):
        dosya_yolu = os.path.join(okra_klasoru, dosya)
        gorsel = load_and_blur_image(dosya_yolu)
        okra_gorselleri.append(gorsel)

# Normal dosyaları oku ve gorselleri normal_gorselleri listesine ekle
for dosya in os.listdir(normal_klasoru):
    if dosya.endswith('.png'):
        dosya_yolu = os.path.join(normal_klasoru, dosya)
        gorsel = load_and_blur_image(dosya_yolu)
        normal_gorselleri.append(gorsel)

# Okra ve normal gorsellerini birleştir
okra_gorselleri = np.array(okra_gorselleri)
normal_gorselleri = np.array(normal_gorselleri)
X = np.concatenate((okra_gorselleri, normal_gorselleri))

# Etiketleri oluştur
okra_etiketleri = np.ones(len(okra_gorselleri))
normal_etiketleri = np.zeros(len(normal_gorselleri))
y = np.concatenate((okra_etiketleri, normal_etiketleri))

# Verileri standartlaştırma
scaler = StandardScaler()
X = scaler.fit_transform(X.reshape(X.shape[0], -1))
X = X.reshape(X.shape[0], 1, 512*512)  # LSTM giriş şekline uyacak şekilde yeniden şekillendir

# Verileri eğitim ve test setlerine ayırma
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# K-fold cross validation için k değerini belirleyin
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Ortalama değerleri saklamak için boş listeler oluşturun
ortalama_kayip = []
ortalama_dogruluk = []

# Modelinizi her bir fold için eğitin ve değerlendirin
for i, (train_index, val_index) in enumerate(kf.split(X)):
    print(f"Fold {i+1}/{k}")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Modeli oluşturun
    model = Sequential([
        LSTM(128, input_shape=(1, 512*512), return_sequences=True),
        Dropout(0.5),
        LSTM(64, return_sequences=False),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    # Modeli derleme
    optimizer = Adam(learning_rate=0.0005)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping tanımlama
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Modeli eğitme ve early stopping kullanma
    model.fit(X_train, y_train, epochs=40, batch_size=8, validation_data=(X_val, y_val), callbacks=[early_stopping])

    # Model performansını değerlendirme
    loss, accuracy = model.evaluate(X_test, y_test)
    ortalama_kayip.append(loss)
    ortalama_dogruluk.append(accuracy)

# Sonuçları ortalama olarak değerlendirin
print(f"Ortalama kayıp: {np.mean(ortalama_kayip)}")
print(f"Ortalama doğruluk: {np.mean(ortalama_dogruluk)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fold 1/5
Epoch 1/40
100/100 [==============================] - 18s 111ms/step - loss: 0.6782 - accuracy: 0.5877 - val_loss: 0.6522 - val_accuracy: 0.6900
Epoch 2/40
100/100 [==============================] - 9s 92ms/step - loss: 0.6679 - accuracy: 0.6065 - val_loss: 0.6405 - val_accuracy: 0.6800
Epoch 3/40
100/100 [==============================] - 9s 94ms/step - loss: 0.6463 - accuracy: 0.6378 - val_loss: 0.6266 - val_accuracy: 0.6600
Epoch 4/40
100/100 [==============================] - 9s 91ms/step - loss: 0.6414 - accuracy: 0.6504 - val_loss: 0.6205 - val_accuracy: 0.6750
Epoch 5/40
100/100 [==============================] - 9s 86ms/step - loss: 0.6352 - accuracy: 0.6516 - val_loss: 0.6219 - val_accuracy: 0.6650
Epoch 6/40
100/100 [==============================] - 10s 98ms/step - loss: 0.6320 - accuracy: 0.6441 - val_loss: 0.6132 - val_accuracy: 0.6850
E

# CNN

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Veri klasörlerinin yolu
normal_klasoru = "/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Saglikli_Grafikler_Siyah_Beyaz_1024x1024"
okra_klasoru = "/content/drive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Okrali_Grafikler_Siyah_Beyaz_1024x1024"

# Okra ve normal verilerini saklamak için boş liste oluştur
okra_gorselleri = []
normal_gorselleri = []

# Okra dosyalarını oku ve gorselleri okra_gorselleri listesine ekle
for dosya in os.listdir(okra_klasoru):
    if dosya.endswith('.png'):
        dosya_yolu = os.path.join(okra_klasoru, dosya)
        gorsel = load_img(dosya_yolu, color_mode='grayscale', target_size=(512, 512))
        gorsel = img_to_array(gorsel)
        okra_gorselleri.append(gorsel)

# Normal dosyaları oku ve gorselleri normal_gorselleri listesine ekle
for dosya in os.listdir(normal_klasoru):
    if dosya.endswith('.png'):
        dosya_yolu = os.path.join(normal_klasoru, dosya)
        gorsel = load_img(dosya_yolu, color_mode='grayscale', target_size=(512, 512))
        gorsel = img_to_array(gorsel)
        normal_gorselleri.append(gorsel)

# Okra ve normal gorsellerini birleştir
okra_gorselleri = np.array(okra_gorselleri)
normal_gorselleri = np.array(normal_gorselleri)
X = np.concatenate((okra_gorselleri, normal_gorselleri))

# Etiketleri oluştur
okra_etiketleri = np.ones(len(okra_gorselleri))
normal_etiketleri = np.zeros(len(normal_gorselleri))
y = np.concatenate((okra_etiketleri, normal_etiketleri))

# Verileri eğitim ve test setlerine ayır
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Model oluşturma
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Modeli derleme
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğitme
model.fit(X_egitim, y_egitim, epochs=50, batch_size=32, validation_split=0.2)

# Modeli değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test setinde kayıp: {loss}")
print(f"Test setinde doğruluk: {accuracy}")


Epoch 1/50
20/20 [==============================] - 21s 557ms/step - loss: 278.1176 - accuracy: 0.4859 - val_loss: 0.6797 - val_accuracy: 0.5500
Epoch 2/50
20/20 [==============================] - 4s 215ms/step - loss: 3.0181 - accuracy: 0.6034 - val_loss: 0.6529 - val_accuracy: 0.6438
Epoch 3/50
20/20 [==============================] - 4s 212ms/step - loss: 0.6891 - accuracy: 0.5031 - val_loss: 0.6936 - val_accuracy: 0.4500
Epoch 4/50
20/20 [==============================] - 5s 231ms/step - loss: 0.6931 - accuracy: 0.5031 - val_loss: 0.6937 - val_accuracy: 0.4500
Epoch 5/50
20/20 [==============================] - 4s 219ms/step - loss: 0.6931 - accuracy: 0.5031 - val_loss: 0.6937 - val_accuracy: 0.4500
Epoch 6/50
20/20 [==============================] - 4s 215ms/step - loss: 0.6931 - accuracy: 0.5031 - val_loss: 0.6937 - val_accuracy: 0.4500
Epoch 7/50
20/20 [==============================] - 4s 217ms/step - loss: 0.6931 - accuracy: 0.5031 - val_loss: 0.6937 - val_accuracy: 0.4500
Epo